First we need to load the sequence file

In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
from datasets import load_dataset

trainFile="Data/s288c.fasta.train.txt"
testFile="Data/s288c.fasta.test.txt"
    

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Then we will need to convert each sequence to codons, each codon is treated as a "word" and the sequence is the "sentence"

In [3]:
dataset=load_dataset('text', data_files=trainFile,split='train')



Using custom data configuration default-00ee6744db450d6f
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-00ee6744db450d6f/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


Now let's try to train a tokenlizer on our data

In [4]:
dataset

Dataset({
    features: ['text'],
    num_rows: 4990
})

In [5]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

In [6]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

This is where the tokenizer starts:

In [7]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

In [8]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=False)


In [9]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(),normalizers.StripAccents()]
)

In [10]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()


In [11]:
tokenizer.pre_tokenizer.pre_tokenize_str("CAA GTG ACC AGA ATG ATC ACC GGT GTT CAA CAA")


[('CAA', (0, 3)),
 ('GTG', (4, 7)),
 ('ACC', (8, 11)),
 ('AGA', (12, 15)),
 ('ATG', (16, 19)),
 ('ATC', (20, 23)),
 ('ACC', (24, 27)),
 ('GGT', (28, 31)),
 ('GTT', (32, 35)),
 ('CAA', (36, 39)),
 ('CAA', (40, 43))]

In [12]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=5000, special_tokens=special_tokens)

In [13]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)


In [14]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [15]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [16]:
encoding = tokenizer.encode("CAA GTG ACC AGA ATG ATC ACC GGT GTT")


In [17]:
encoding.tokens

['[CLS]',
 'CAA',
 'GTG',
 'ACC',
 'AGA',
 'ATG',
 'ATC',
 'ACC',
 'GGT',
 'GTT',
 '[SEP]']

In [18]:
encoding.type_ids


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [19]:
tokenizer.decoder = decoders.WordPiece(prefix="##")
from transformers import BertTokenizerFast
tokenizer.save("./yeast_token")
tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)


In [20]:
from transformers import BertTokenizer
from transformers import AutoTokenizer




In [21]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [22]:
datasets = load_dataset("text", data_files={"train": trainFile, "validation": testFile})
print(datasets)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(tokenized_datasets)

Using custom data configuration default-69e50c05f0cb0ce3
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})
 

Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-1bfea4b7eceb98e8.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-79ef0d0ff506e9c9.arrow


...

Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-91820736caeca892.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6f7b03fe596b7234.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-571414735160276b.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e92b2a039292720f.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})


In [23]:
import pandas as pd
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(examples["text"])

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# block_size = tokenizer.model_max_length
block_size = 128

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

print(tokenized_datasets)
print(tokenized_datasets['train'][1])

Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-1bfea4b7eceb98e8.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-79ef0d0ff506e9c9.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4628096e46f813b1.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-621fee93fb12bd8f.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-91820736caeca892.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-6f7b03fe596b7234.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-571414735160276b.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-e92b2a039292720f.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})
{'input_ids': [2, 55, 48, 34, 80, 30, 42, 50, 54, 30, 23, 44, 46, 25, 34, 66, 56, 43, 53, 66, 61, 74, 31, 51, 55, 23, 53, 59, 43, 43, 37, 34, 81, 57, 52, 33, 66, 51, 54, 23, 74, 53, 23, 63, 48, 53, 47, 23, 43, 40, 31, 52, 23, 34, 23, 34, 47, 71, 61, 43, 43, 47, 53, 49, 23, 55, 85, 25, 66, 81, 35, 34, 65, 30, 50, 47, 59, 80, 28, 31, 27, 75, 36, 74, 81, 50, 78, 64, 27, 70, 64, 25, 44, 28, 72, 48, 80, 45, 34, 63, 54, 42, 65, 69, 71, 71, 58, 31, 33, 55, 71, 36, 45, 73, 61, 28, 61, 28, 27, 85, 62, 45, 28, 43, 69, 45, 23, 27, 72, 50, 30, 28, 46, 33, 40, 67, 25, 45, 30, 45, 88, 23, 72, 77, 25, 36, 47, 55, 30, 30, 25, 80, 46, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
lm_datasets =tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)



Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-ac641f594b76cc45.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-2de19c2fac7eadb4.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-3443e2c1e6684330.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-72e4db1dade390c0.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4e1164e958248f8d.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-ec14f6e92845714b.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-0824f169e9289d29.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-69e50c05f0cb0ce3/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-54b62b79af798139.arrow


In [25]:
lm_datasets["train"][1]



{'input_ids': [61,
  28,
  34,
  40,
  81,
  35,
  27,
  27,
  61,
  61,
  23,
  72,
  63,
  23,
  31,
  31,
  25,
  47,
  52,
  81,
  40,
  38,
  23,
  34,
  33,
  28,
  57,
  52,
  65,
  30,
  49,
  71,
  23,
  27,
  27,
  54,
  30,
  45,
  50,
  54,
  25,
  61,
  33,
  47,
  41,
  27,
  37,
  31,
  61,
  81,
  45,
  31,
  25,
  40,
  85,
  41,
  33,
  25,
  41,
  74,
  23,
  64,
  36,
  25,
  36,
  42,
  49,
  23,
  25,
  56,
  61,
  23,
  27,
  70,
  25,
  88,
  57,
  54,
  25,
  31,
  34,
  63,
  36,
  23,
  37,
  41,
  36,
  25,
  42,
  27,
  31,
  23,
  30,
  34,
  38,
  40,
  40,
  23,
  71,
  33,
  42,
  35,
  67,
  31,
  40,
  49,
  48,
  78,
  41,
  33,
  46,
  45,
  27,
  41,
  47,
  49,
  52,
  27,
  23,
  61,
  48,
  38,
  47,
  36,
  27,
  41,
  30,
  33],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [26]:
torch.cuda.memory_summary(device=None, abbreviated=False)
torch.cuda.empty_cache()


In [27]:
print(lm_datasets["train"])
print(lm_datasets["train"][0])


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 19207
})
{'input_ids': [2, 56, 27, 25, 37, 56, 81, 54, 45, 54, 41, 73, 45, 27, 25, 78, 25, 53, 25, 30, 78, 81, 33, 46, 78, 25, 81, 76, 78, 80, 41, 70, 25, 44, 40, 25, 34, 78, 31, 23, 69, 44, 53, 44, 48, 25, 54, 55, 52, 31, 58, 23, 54, 36, 78, 31, 40, 78, 40, 31, 86, 41, 30, 25, 78, 53, 35, 27, 45, 31, 33, 31, 54, 28, 36, 53, 44, 28, 34, 23, 67, 59, 66, 44, 79, 81, 57, 38, 67, 28, 56, 52, 25, 71, 79, 42, 36, 53, 51, 53, 42, 85, 40, 33, 69, 64, 40, 34, 69, 40, 65, 37, 40, 31, 40, 30, 38, 65, 67, 36, 25, 54, 34, 67, 40, 25, 33, 25], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [28]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"CUBERT",
    evaluation_strategy = "epoch",
    learning_rate=1e-3,
    weight_decay=0.01,
    num_train_epochs=1,
)

from transformers import AutoConfig, AutoModelForCausalLM
model_checkpoint = "bert-base-uncased"

config = AutoConfig.from_pretrained(model_checkpoint)

model = AutoModelForCausalLM.from_config(config)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [29]:
import torch
print(torch.cuda.is_available())


True


In [30]:
print(model)

BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [31]:
trainer.train()

/home/lu/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 19207
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2401


Epoch,Training Loss,Validation Loss
1,3.963500,3.963619


Saving model checkpoint to CUBERT/checkpoint-500
Configuration saved in CUBERT/checkpoint-500/config.json
Model weights saved in CUBERT/checkpoint-500/pytorch_model.bin
Saving model checkpoint to CUBERT/checkpoint-1000
Configuration saved in CUBERT/checkpoint-1000/config.json
Model weights saved in CUBERT/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to CUBERT/checkpoint-1500
Configuration saved in CUBERT/checkpoint-1500/config.json
Model weights saved in CUBERT/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to CUBERT/checkpoint-2000
Configuration saved in CUBERT/checkpoint-2000/config.json
Model weights saved in CUBERT/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3679
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2401, training_loss=3.979353136144445, metrics={'train_runtime': 1587.6055, 'train_samples_per_second': 12.098, 'train_steps_per_second': 1.512, 'total_flos': 1263843709862400.0, 'train_loss': 3.979353136144445, 'epoch': 1.0})

In [32]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 3679
  Batch size = 8


NameError: name 'math' is not defined